In [11]:
import json
import re
import pickle

import requests
import pandas as pd

### 演示Github媒体API使用方法

In [2]:
github_path = 'https://api.github.com/repos/{owner}/{repo}'
Accept = 'application/vnd.github.v3.star+json'
github_app_res = requests.get(github_path.format(owner='ipython', repo='ipython'), headers={'Accept': Accept})
github_app_info = json.loads(github_app_res.text)
github_app_info

{'id': 658518,
 'node_id': 'MDEwOlJlcG9zaXRvcnk2NTg1MTg=',
 'name': 'ipython',
 'full_name': 'ipython/ipython',
 'private': False,
 'owner': {'login': 'ipython',
  'id': 230453,
  'node_id': 'MDEyOk9yZ2FuaXphdGlvbjIzMDQ1Mw==',
  'avatar_url': 'https://avatars3.githubusercontent.com/u/230453?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/ipython',
  'html_url': 'https://github.com/ipython',
  'followers_url': 'https://api.github.com/users/ipython/followers',
  'following_url': 'https://api.github.com/users/ipython/following{/other_user}',
  'gists_url': 'https://api.github.com/users/ipython/gists{/gist_id}',
  'starred_url': 'https://api.github.com/users/ipython/starred{/owner}{/repo}',
  'subscriptions_url': 'https://api.github.com/users/ipython/subscriptions',
  'organizations_url': 'https://api.github.com/users/ipython/orgs',
  'repos_url': 'https://api.github.com/users/ipython/repos',
  'events_url': 'https://api.github.com/users/ipython/events{/privacy}',
  'rece

In [3]:
star_path = 'https://api.github.com/repos/{owner}/{repo}/stargazers'
Accept = 'application/vnd.github.v3.star+json'
star_res = requests.get(star_path.format(owner='ipython', repo='ipython'), headers={'Accept': Accept})
star_info = json.loads(star_res.text)
star_info

[{'starred_at': '2010-05-10T04:46:06Z',
  'user': {'login': 'kanzure',
   'id': 101238,
   'node_id': 'MDQ6VXNlcjEwMTIzOA==',
   'avatar_url': 'https://avatars1.githubusercontent.com/u/101238?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/kanzure',
   'html_url': 'https://github.com/kanzure',
   'followers_url': 'https://api.github.com/users/kanzure/followers',
   'following_url': 'https://api.github.com/users/kanzure/following{/other_user}',
   'gists_url': 'https://api.github.com/users/kanzure/gists{/gist_id}',
   'starred_url': 'https://api.github.com/users/kanzure/starred{/owner}{/repo}',
   'subscriptions_url': 'https://api.github.com/users/kanzure/subscriptions',
   'organizations_url': 'https://api.github.com/users/kanzure/orgs',
   'repos_url': 'https://api.github.com/users/kanzure/repos',
   'events_url': 'https://api.github.com/users/kanzure/events{/privacy}',
   'received_events_url': 'https://api.github.com/users/kanzure/received_events',
   'type': 'Us

In [5]:
star_res.headers['link']

'<https://api.github.com/repositories/658518/stargazers?page=2>; rel="next", <https://api.github.com/repositories/658518/stargazers?page=484>; rel="last"'

### 开始爬取

In [6]:
df = pd.read_pickle('pypi_json.pkl')

In [7]:
for i in df.github:
    if i not in (None, True, False):
        print(type(i))
        break

<class 'tuple'>


In [8]:
df

name  python2  python3 python  \
hdfdict     0.1.1alpha      hdfdict    False    False     []   
            0.1alpha        hdfdict    False    False     []   
            0.3.1           hdfdict    False     True  [3.7]   
hdfe        0.0.3              hdfe    False     True     []   
            0.0.4              hdfe    False     True     []   
...                             ...      ...      ...    ...   
auto_tagify 1.0         auto_tagify    False    False     []   
            1.1         auto_tagify    False    False  [2.6]   
            1.2         auto_tagify    False    False     []   
            1.3         auto_tagify    False    False     []   
            1.4         auto_tagify    False    False     []   

                                                   topic  \
hdfdict     0.1.1alpha                                {}   
            0.1alpha                                  {}   
            0.3.1                                     {}   
hdfe        0.0.3                                     {}   
            0.0.4                                     {}   
...                                                  ...   
auto_tagify 1.0                                       {}   
            1.1                                       {}   
            1.2         {Internet, Software Development}   
            1.3         {Internet, Software Development}   
            1.4         {Internet, Software Development}   

                                    dependency         dependency_detail  \
hdfdict     0.1.1alpha           [h5py, numpy]              [None, None]   
            0.1alpha                        []                        []   
            0.3.1       [h5py-wrapper, pyyaml]  [>=1.1,<2.0, >=5.1,<6.0]   
hdfe        0.0.3       [numpy, pandas, scipy]    [None, >=0.25.0, None]   
            0.0.4       [numpy, pandas, scipy]    [None, >=0.25.0, None]   
...                                        ...                       ...   
auto_tagify 1.0                             []                        []   
            1.1                             []                        []   
            1.2                             []                        []   
            1.3                             []                        []   
            1.4                             []                        []   

                       requires_python  \
hdfdict     0.1.1alpha            None   
            0.1alpha              None   
            0.3.1           >=3.7,<4.0   
hdfe        0.0.3                        
            0.0.4                        
...                                ...   
auto_tagify 1.0                   None   
            1.1                   None   
            1.2                   None   
            1.3                   None   
            1.4                   None   

                                                             project_urls  \
hdfdict     0.1.1alpha  {'Homepage': 'https://github.com/SiggiGue/hdfd...   
            0.1alpha       {'Download': 'UNKNOWN', 'Homepage': 'UNKNOWN'}   
            0.3.1       {'Homepage': 'https://github.com/SiggiGue/hdfd...   
hdfe        0.0.3       {'Homepage': 'https://github.com/esantorella/h...   
            0.0.4       {'Homepage': 'https://github.com/esantorella/h...   
...                                                                   ...   
auto_tagify 1.0         {'Download': 'UNKNOWN', 'Homepage': 'http://gi...   
            1.1         {'Download': 'UNKNOWN', 'Homepage': 'http://gi...   
            1.2         {'Download': 'UNKNOWN', 'Homepage': 'https://g...   
            1.3         {'Download': 'UNKNOWN', 'Homepage': 'https://g...   
            1.4         {'Download': 'UNKNOWN', 'Homepage': 'https://g...   

                       upload_time                      github  
hdfdict     0.1.1alpha  2015-02-27         (SiggiGue, hdfdict)  
            0.1alpha    2015-02-27                        True  
   

In [14]:
def get_star_time(df):
    star_path = 'https://api.github.com/repos/{}/{}/stargazers'
    token = 'a778aeacfadcaae486f97b8c9a560121b4b44711'
    Accept = 'application/vnd.github.v3.star+json'
    star_infos = []
    for package_name, aha in df.github.groupby(level=0):
        for i in aha.values[::-1]:
            if i not in (None, True, False):
                star_res = requests.get(star_path.format(*i), headers={'Accept': Accept, 'Authorization': 'token ' + token})
                msg = json.loads(star_res.text)
                try:
                    star_infos.append(dict(
                        _id=package_name,
                        star_time=[i['starred_at'] for i in msg]
                    ))
                except TypeError:
                    if msg['message'] == 'Not Found':
                        continue
                    elif 'rate limit' in msg['message']:
                        return star_infos
                    else:
                        print(msg)
                        return star_infos
                except Exception as e:
                    print(e)
                else:
                    break
    return star_infos

In [10]:
packages = sorted(set(list(zip(*df.index))[0]))

In [20]:
star_infos = get_star_time(df.loc[packages[packages.index(star_infos[-1]['_id']):20000]])

In [23]:
packages.index(star_infos[-1]['_id'])

12230

In [22]:
with open(f"./STAR/star_info_10000-{packages.index(star_infos[-1]['_id'])}", 'wb') as f:
    pickle.dump(star_infos, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
star_path = 'https://api.github.com/repos/{}/{}/stargazers'
token = 'a778aeacfadcaae486f97b8c9a560121b4b44711'
Accept = 'application/vnd.github.v3.star+json'
i = 'ipython', 'ipython'
res = requests.get(star_path.format(*i), headers={'Accept': Accept, 'Authorization': 'token ' + token})

In [8]:
res.headers['X-RateLimit-Remaining']

'4268'